### MDR的平均曲率流演化

* 通过MDR的切向速度 

\begin{align*}
-\Delta_{\Gamma}\mathbf v = \kappa n^{old} \quad \text{on }\Gamma\\
\mathbf v\cdot {n}^{old} = -H^{old} \quad \text{on }\Gamma
\end{align*}

In [1]:
from ngsolve import *

In [2]:
# from netgen.csg import Pnt,SplineCurve2d,CSGeometry,Revolution,Sphere
import numpy as np
import netgen.meshing as ngm
from netgen.csg import *
from netgen.meshing import MeshingStep
from ngsolve.comp import IntegrationRuleSpaceSurface

In [3]:
from geometry import *

In [4]:
from ngsolve.webgui import Draw

In [5]:
from netgen.occ import SplineApproximation, Pnt, Axis, Face, Wire, Segment, Revolve, OCCGeometry, Z, X, Y
#%% Construction of Initial Curved Mesh
dim = 3
order = 1
msize = 0.15
dt = Parameter(0)

In [6]:
from ngsolve import Mesh

In [7]:
from esfem import MCF_MDR

### Dumbbell网格生成

#### 需要旋转曲面对象Param2dRot(spline_obj,axis_opt="x",c_tag="True")来设置初始曲面的曲率和法向量

* **需要提供母线的样条对象**
* 该对象需要可以通过坐标反解参数：

In [8]:
maxh = 0.05
mesh, spline_obj = Mesh2dDumbbell(maxh,order,a=0.6,b=0.4,N_Spline=7,eps=2e-7)

tstart (must be vertically): (-3.59866e-07, 0.53507)
tend (must be vertically): (3.59865e-07, -0.53507)


In [9]:
myparam = Param2dRot(spline_obj,axis_opt="x",c_tag="True")

### 几何设置之后开始MCF的演化设置

In [10]:
tauval = 1e-3
T_end = 0.43
order = 2
MCF_MDR_Obj = MCF_MDR(mesh,T=T_end,dt=tauval,order=order)
t = Parameter(0)

fes ndofs is 11246


In [11]:
print(MCF_MDR_Obj.T)

0.43


In [12]:
MCF_MDR_Obj.IniByGeoObj(myparam)

L2 Error of position on modified surface is 0.010053252684539925


In [13]:
import ipywidgets as widgets
from IPython.display import display
import time
step = 0
import copy

In [14]:
MCF_MDR_Obj.WeakMCF()

In [15]:
sceneu = Draw(x, mesh, 'disp')
t_old = 0.0
pbar = widgets.FloatProgress(
    value=0.0, min=0.0, max=1,
    description='t:', bar_style='info',
    layout=widgets.Layout(width='600px')
)
info = widgets.HTML(value="")
display(widgets.VBox([pbar, info]))

t0 = time.time()

while t_old<MCF_MDR_Obj.T:
    tauval = MCF_MDR_Obj.dt.Get()
    #     if t_old > 0.3:
    #         tauval = MCF_MDR_Obj.Get_Proper_dt()
    #     MCF_MDR_Obj.ReIni_HN_ByDisPos('BGN',0.1)
    MCF_MDR_Obj.lhs.Assemble()
    MCF_MDR_Obj.rhs.Assemble()
    MCF_MDR_Obj.gfu.vec.data = MCF_MDR_Obj.lhs.mat.Inverse(inverse="umfpack")*MCF_MDR_Obj.rhs.vec

    MCF_MDR_Obj.gfuold.vec.data = BaseVector(copy.deepcopy(MCF_MDR_Obj.gfu.vec.FV().NumPy())) 
    MCF_MDR_Obj.Disp.vec.data += BaseVector(tauval*MCF_MDR_Obj.gfu.components[3].vec.FV().NumPy())
    MCF_MDR_Obj.mesh.SetDeformation(MCF_MDR_Obj.Disp)
    t_old += tauval    
    sceneu.Redraw()
    
    # update progress UI
    pbar.value = min(t_old, 1)
    elapsed = time.time() - t0
    info.value = f"<b>step</b>={step} &nbsp; <b>t</b>={t_old:.6f}/{T_end} &nbsp; <b>elapsed</b>={elapsed:.1f}s"

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.23…

KeyboardInterrupt: 